# Testing `numpy` inference

### Imports

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import sys
import time
import pickle

#import numpy as np
import autograd.numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=3, linewidth=500, edgeitems=15, suppress=True)

import seaborn as sns
sns.set_palette('bright')


import matplotlib as mpl
import matplotlib.style as mplstyle

label_size = 9 
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

mplstyle.use('fast')

#from matplotlib import rc
#rc('text', usetex=False)

import utils.file_io as io

/Users/saforem2/opt/anaconda3/envs/tf12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/saforem2/opt/anaconda3/envs/tf12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/saforem2/opt/anaconda3/envs/tf12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/saforem2/opt/anaconda3/e

In [3]:
from plotters.plot_observables import grid_plot, get_obs_dict
import utils.file_io as io

import pandas as pd
from plotters.plot_utils import load_pkl

from plotters.plot_observables import get_run_dirs, get_title_str
from lattice.lattice import u1_plaq_exact
import scipy
import datetime
import matplotlib as mpl
label_size = 9
mpl.rcParams['xtick.labelsize'] = label_size 
mpl.rcParams['ytick.labelsize'] = label_size 

In [4]:
from params.gauge_params import GAUGE_PARAMS
from trainers.train_setup import train_setup
from utils.attr_dict import AttrDict
from models.gauge_model import GaugeModel

## Create `GaugeModel`

In [5]:
from trainers.train_setup import train_setup
from params.gauge_params import GAUGE_PARAMS

_params = GAUGE_PARAMS.copy()
params, hooks = train_setup(_params, log_file=None)
params['zero_masks'] = False
params['space_size'] = 8
params['time_size'] = 8
params['batch_size'] = 64 
params['train_steps'] = 100
for key, val in params.items():
    print(f'{key}: {val}')

--------------------------------------------------------------------------------
Starting training using L2HMC algorithm...
Creating directory: /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224
space_size: 8
time_size: 8
link_type: U1
dim: 2
batch_size: 64
rand: True
num_steps: 5
eps: 0.2
fixed_beta: False
beta_init: 2.0
beta_final: 5.0
lr_init: 0.001
lr_decay_steps: 1000
lr_decay_rate: 0.96
warmup_lr: False
train_steps: 100
save_steps: 1000
logging_steps: 2500
print_steps: 1
network_arch: generic
num_hidden1: 50
num_hidden2: 50
use_bn: False
dropout_prob: 0.0
clip_value: 0.0
summaries: True
eps_fixed: False
hmc: False
use_nnehmc_loss: False
use_gaussian_loss: False
loss_scale: 1
std_weight: 1.0
aux_weight: 1.0
charge_weight: 0.0
metric: cos_diff
x_scale_weight: 1
x_translation_weight: 1
x_transformation_weight: 1
v_scale_weight: 1
v_translation_weight: 1
v_transformation_weight: 1
log_dir: /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224
tr

1. Create `GaugeModel`
2. Create `TrainLogger`
3. Create `tf.ConfigProto()`
4. Create `tf.Session`
5. Create `Trainer`
6. Run `Trainer` on `GaugeModel` to train the model

In [6]:
from models.gauge_model import GaugeModel
model = GaugeModel(params)

--------------------------------------------------------------------------------
INFO: Building graph for `GaugeModel`...
INFO: Creating lattice...
INFO: Creating input placeholders...
x: Tensor("GaugeModel/inputs/x:0", shape=(64, 128), dtype=float32)

beta: Tensor("GaugeModel/inputs/beta:0", shape=(), dtype=float32)

eps_ph: Tensor("GaugeModel/inputs/eps_ph:0", shape=(), dtype=float32)

global_step_ph: Tensor("GaugeModel/inputs/global_step_ph:0", shape=(), dtype=int64)

train_phase: Tensor("GaugeModel/inputs/is_training:0", shape=(), dtype=bool)

net_weights: NetWeights(x_scale=<tf.Tensor 'GaugeModel/inputs/x_scale_weight:0' shape=() dtype=float32>, x_translation=<tf.Tensor 'GaugeModel/inputs/x_translation_weight:0' shape=() dtype=float32>, x_transformation=<tf.Tensor 'GaugeModel/inputs/x_transformation_weight:0' shape=() dtype=float32>, v_scale=<tf.Tensor 'GaugeModel/inputs/v_scale_weight:0' shape=() dtype=float32>, v_translation=<tf.Tensor 'GaugeModel/inputs/v_translation_weight:0' 

In [7]:
from loggers.train_logger import TrainLogger
from trainers.train_setup import create_config

train_logger = TrainLogger(model, model.log_dir,
                           logging_steps=model.logging_steps,
                           summaries=params['summaries'])
config, params = create_config(params)

sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

Creating directory: /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224/checkpoints
Creating directory: /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224/training
Creating directory: /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224/summaries/train


In [8]:
from trainers.trainer import Trainer
trainer = Trainer(sess, model, train_logger, **params)

In [9]:
from config import NetWeights, NP_FLOAT

samples_init = np.array(model.lattice.samples_array, dtype=NP_FLOAT)

net_weights_init = NetWeights(1, 1, 1, 1, 1, 1)

trainer.train(model.train_steps,
              beta=model.beta_init,
              samples=samples_init,
              net_weights=net_weights_init)

Global step: 0

-------------------------------------------------------------------------------------------------------
    STEP      t/STEP     LOSS     % ACC      EPS        dX       BETA       LR      ACTION    dPLAQ   
-------------------------------------------------------------------------------------------------------
    0/100      12.51    -110.8    0.8111     0.201    0.4991       2       0.001     63.25    0.6861   
    1/100     0.1405     -98.1    0.6945     0.202    0.4354     2.012     0.001     43.64    0.3816   
    2/100     0.1478    -90.23    0.5752     0.203    0.4104     2.024     0.001     35.92     0.263   
    3/100     0.1482    -87.96    0.5417     0.204    0.4012     2.037     0.001     31.77    0.2002   
    4/100     0.1698    -84.59    0.4779     0.205    0.3989     2.049     0.001     29.35    0.1643   
    5/100     0.1667    -88.54    0.5987     0.206    0.3848     2.062     0.001     27.76    0.1415   
    6/100     0.1555    -96.73    0.6861    0.206

   76/100     0.1818    -107.5    0.4555    0.2586    0.4022     3.676     0.001     10.74    0.01755  
   77/100     0.1511    -105.6    0.4857    0.2588    0.4009     3.717     0.001     10.61    0.01749  
   78/100     0.1512    -105.6    0.4677    0.2585    0.4023     3.759     0.001     10.45    0.0168   
   79/100     0.1615    -112.1    0.4855    0.2578    0.4046     3.802     0.001     10.29    0.01632  
   80/100     0.1774    -103.2    0.4786    0.2572    0.3987     3.846     0.001     10.17    0.01625  
   81/100     0.1564    -97.65     0.338    0.2565    0.4016     3.891     0.001     10.14    0.01758  
   82/100     0.1638    -100.4    0.4507    0.2557    0.4063     3.937     0.001     10.21    0.02056  
   83/100     0.1479    -90.55    0.3553    0.2545    0.3963     3.984     0.001     9.945    0.0183   
   84/100     0.1793    -107.2    0.5037    0.2534    0.3973     4.032     0.001     9.944    0.02013  
   85/100     0.1644    -99.85    0.4328    0.2525    0.3925    

 - Save weights from the trained model to `.pkl` file:

In [10]:
def pkl_dump(obj, out_file, name=''):
    io.log(f'Saving {name} to {out_file}')
    with open(out_file, 'wb') as f:
        pickle.dump(obj, f)

In [11]:
from trainers.train_setup import get_net_weights

#wfile = os.path.join(model.log_dir, 'dynamics_weights.h5')
#model.dynamics.save_weights(wfile)

weights_final, coeffs_final = get_net_weights(model, sess)
xcoeffs = sess.run(list(coeffs_final['xnet'].values()))
vcoeffs = sess.run(list(coeffs_final['vnet'].values()))
weights_final['xnet']['GenericNet'].update({
    'coeff_scale': xcoeffs[0],
    'coeff_transformation': xcoeffs[1],
})
weights_final['vnet']['GenericNet'].update({
    'coeff_scale': vcoeffs[0],
    'coeff_transformation': vcoeffs[1],
})
pkl_dump(weights_final, os.path.join(model.log_dir, 'weights.pkl'), name='weights_final')

Saving weights_final to /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224/weights.pkl


In [23]:
masks_file = os.path.join(model.log_dir, 'dynamics_mask.pkl')
m_ = sess.run(model.dynamics.masks)
pkl_dump(m_, masks_file)

Saving  to /Users/saforem2/ANL/l2hmc-qcd/gauge_logs/2020_01_27/L8_b64_lf5_f32_0224/dynamics_mask.pkl


 * Create `dynamics_np` to compare against `model.dynamics`:

In [25]:
from runners.runner_np import create_dynamics, load_pkl, run_inference_np

eps_np = sess.run(model.dynamics.eps)
dynamics_np, lattice = create_dynamics(model.log_dir, eps=eps_np,
                                       num_steps=model.num_steps,
                                       batch_size=model.batch_size)
with open(masks_file, 'rb') as f:
    _m = pickle.load(f)
    
dynamics_np.masks = _m

Create operations for calculating quantities of interest:   
 - `xf`: Proposed $x$, obtained from running `model.dyanmics.transition_kernel` with `forward=True`   
 - `vf`: Proposed $v$, obtained from running `model.dynamics.transition_kernel` with `forward=True`
 - `pxf`: $A(\xi^{\prime}|\xi)$
 - `sumlogdetf`: Log determinant, accumulated over all leapfrog steps. Given by:
 
 \begin{equation}
 \log|\mathcal{J}| = \log\left|\frac{\partial\xi^{\prime}}{\partial \xi^{T}}\right| = d \sum_{t\leq N_{\mathrm{LF}}} \left[\frac{\varepsilon}{2}\mathbb{1} \cdot S_{v}(\zeta^{t}_{1}) + \varepsilon m^{t} \cdot S_{x}(\zeta^{t}_{2}) + \varepsilon\bar{m}^{t} \cdot S_{x}(\zeta^{t}_{3}) + \frac{\varepsilon}{2}\mathbb{1}\cdot S_{v}(\zeta^{t}_{4})\right]
 \label{eq:sumlogdet}
 \end{equation}   
 
- `xf_`: $x^{\prime\prime}$, obtained by updating $x$ for a single leapfrog step.
- `vf_`: $v^{\prime\prime}$, obtained by updating $v$ for a single leapfrog step.
- `sld_`: Accumulated log determinant after a single leapfrog step. ($t = 1$ in Eq.\ref{eq:sumlogdet})
- `dudx_tf_`: $\partial_{x}U(x, \beta)$, evaluated at $\beta = 5$

In [27]:
from config import State, TF_FLOAT
from seed_dict import seeds

# ------------------------------
# Run `model.dynamics` forward:
# ------------------------------
#vf_init = tf.random_normal(tf.shape(model.x), dtype=TF_FLOAT, seed=seeds['vf_init'], name='vf_init')
vf_init_np = np.array(np.random.randn(*model.x.shape), dtype=NP_FLOAT)
vf_init = tf.constant(vf_init_np)

state_init_f = State(model.x, vf_init, model.beta)
outf = model.dynamics.transition_kernel(*state_init_f,
                                        model.net_weights,
                                        model.train_phase,
                                        forward=True, hmc=True)
xf = outf['x_proposed']
vf = outf['v_proposed']
pxf = outf['accept_prob']
pxf_hmc = outf['accept_prob_hmc']
sumlogdetf = outf['sumlogdet']

In [28]:
step = 0
# Create operations for getting the output from `model.dynamics._forward_lf`:
xf_, vf_, sld_, _ = model.dynamics._forward_lf(model.x, vf_init,
                                               model.beta, step,
                                               model.net_weights,
                                               training=model.train_phase)

# Create operation for calculating the gradient of the potential
dudx_tf_ = model.dynamics.grad_potential(model.x, model.beta)

In [29]:
train_phase = False
t = model.dynamics._get_time(step, tile=tf.shape(model.x)[0])
Sv, Tv, Qv = model.dynamics.vnet([model.x, dudx_tf_, t], model.train_phase)

mask, mask_inv = model.dynamics._get_mask(step)
Sx, Tx, Qx = model.dynamics.xnet([vf_init, mask * model.x, t], model.train_phase)

 - Run `model.dynamics` forward and compare against results from running `dynamics_np` forward:

In [30]:
step = 0
beta_np = 5.
train_phase = False
net_weights = NetWeights(1, 1, 1, 1, 1, 1)
#xf_init_np = np.zeros(model.x.shape, dtype=NP_FLOAT)
xf_init_np = np.array(np.random.randn(*model.x.shape), dtype=NP_FLOAT)
keys = ['xf', 'vf', 'vf_init', 'pxf', 'sumlogdetf']
fops = [xf, vf, vf_init, pxf, sumlogdetf]
_keys = ['xf_', 'vf_', 'sld_']
_fops = [xf_, vf_, sld_]


def forward_lf_tf(net_weights):
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.net_weights: net_weights,
        model.train_phase: train_phase
    }
    
    _fout = sess.run(_fops, feed_dict=feed_dict)
    _fout_dict = dict(zip(_keys, _fout))
    
    return _fout_dict

    
def forward_lf_np(net_weights, vf_init_np):
    
    _fout_np = dynamics_np._forward_lf(xf_init_np, vf_init_np,
                                       beta_np, step, net_weights)
    _fout_dict_np = dict(zip(_keys, _fout_np))
    return _fout_dict_np


def dynamics_forward_tf(net_weights):
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.net_weights: net_weights,
        model.train_phase: train_phase
    }

    fout = sess.run(fops, feed_dict=feed_dict)
    fout_dict = dict(zip(keys, fout))
    
    return fout_dict


def run_xnet():
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.net_weights: net_weights,
        model.train_phase: train_phase,
    }
    
    outputs = sess.run([Sx, Tx, Qx], feed_dict=feed_dict)
    
    return outputs


def run_vnet():
    feed_dict = {
        model.x: xf_init_np,
        model.beta: beta_np,
        model.train_phase: train_phase,
    }
    outputs = sess.run([Sv, Tv, Qv], feed_dict=feed_dict)
    
    return outputs

def run_xnet_np():
    t = dynamics_np._get_time(step, tile=xf_init_np.shape[0])
    mask, mask_inv = dynamics_np._get_mask(step)
    outputs = dynamics_np.xnet([vf_init_np, mask * xf_init_np, t])
    return outputs


def run_vnet_np():
    t = dynamics_np._get_time(step, tile=xf_init_np.shape[0])
    mask, mask_inv = dynamics_np._get_mask(step)
    dU_dx = dynamics_np.grad_potential(xf_init_np, beta_np)
    outputs = dynamics_np.vnet([xf_init_np, dU_dx, t])
    return outputs
    

def dynamics_forward_np(net_weights, vf_init_np):
    state_init_f_np = State(xf_init_np, vf_init_np, beta_np)
    
    xf, vf, pxf, sldf = dynamics_np.transition_kernel(*state_init_f_np,
                                                      net_weights, forward=True)
    fout_dict_np = {
        'xf': xf,
        'vf': vf,
        'vf_init': vf_init_np,
        'pxf': pxf,
        'sumlogdetf': sldf,
    }
    
    return fout_dict_np


def calc_diff(x1, x2, name=''):
    diff = np.sqrt(np.sum((x1 - x2)**2))
    print(f'{name} diff = {diff}')
    
    return diff

 - Check that $\partial_x U(x)$ is the same for `dynamics_tf` and `dynamics_np`:

In [31]:
xrand_np = np.array(np.random.randn(*xf_init_np.shape), dtype=NP_FLOAT)
dudx_np = dynamics_np.grad_potential(xrand_np, beta_np)

dudx_tf = sess.run(dudx_tf_, feed_dict={model.x: xrand_np, model.beta: beta_np})

diff = np.sqrt(np.sum((dudx_np - dudx_tf) ** 2))
print(f'Gradients agree: {diff < 1e-3}')

Gradients agree: True


In [32]:
Sx_tf, Tx_tf, Qx_tf = run_xnet()
Sx_np, Tx_np, Qx_np = run_xnet_np()
print(f'Sx agrees: {np.allclose(Sx_tf, Sx_np)}')
print(f'Tx agrees: {np.allclose(Tx_tf, Tx_np)}')
print(f'Qx agrees: {np.allclose(Qx_tf, Qx_np)}')

Sv_tf, Tv_tf, Qv_tf = run_vnet()
Sv_np, Tv_np, Qv_np = run_vnet_np()
print(f'Sv agrees: {np.allclose(Sv_tf, Sv_np)}')
print(f'Tv agrees: {np.allclose(Tv_tf, Tv_np)}')
print(f'Qv agrees: {np.allclose(Qv_tf, Qv_np)}')

Sx agrees: True
Tx agrees: True
Qx agrees: True
Sv agrees: False
Tv agrees: False
Qv agrees: False


In [37]:
HEADER = 80 * '-'
net_weights_arr = [
    NetWeights(0, 0, 0, 0, 0, 0),
    #NetWeights(0, 0, 0, 0, 0, 1),
    #NetWeights(0, 0, 0, 0, 1, 0),
    #NetWeights(0, 0, 0, 1, 0, 0),
    #NetWeights(0, 0, 1, 0, 0, 0),
    #NetWeights(0, 1, 0, 0, 0, 0),
    #NetWeights(1, 0, 0, 0, 0, 0),
    NetWeights(1, 1, 1, 1, 1, 1),
]

diffs_dict = {
    
}
for net_weights in net_weights_arr:
    io.log(HEADER)
    io.log(f'NetWeights: {net_weights}')
    fout_tf = dynamics_forward_tf(net_weights)
    diffs_dict[net_weights] = {}
    _vf_np = fout_tf['vf_init']
    np.allclose(_vf_np, vf_init_np)
    fout_np = dynamics_forward_np(net_weights, vf_init_np)
    for key in fout_tf.keys():
        diffs_dict[key] = calc_diff(fout_tf[key], fout_np[key], name=key)
    io.log(HEADER)

--------------------------------------------------------------------------------
NetWeights: NetWeights(x_scale=0, x_translation=0, x_transformation=0, v_scale=0, v_translation=0, v_transformation=0)
xf diff = 1.29953223222401e-05
vf diff = 3.780630140681751e-05
vf_init diff = 0.0
pxf diff = 0.0
sumlogdetf diff = 0.0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
NetWeights: NetWeights(x_scale=1, x_translation=1, x_transformation=1, v_scale=1, v_translation=1, v_transformation=1)
xf diff = 1.858777250163257e-05
vf diff = 5.8459081628825516e-05
vf_init diff = 0.0
pxf diff = 1.3292044968693517e-05
sumlogdetf diff = 3.592385382944485e-06
--------------------------------------------------------------------------------


In [36]:
HEADER1 = (len(f'NetWeights: {tuple(net_weights)}') + 5) * '-'

_diffs_dict = {}
for net_weights in net_weights_arr:
    io.log(HEADER1)
    io.log(f'NetWeights: {tuple(net_weights)}')
    _diffs_dict[net_weights] = {}
    _fout_tf = forward_lf_tf(net_weights)
    #_vf_init = _fout_tf
    _fout_np = forward_lf_np(net_weights, vf_init_np)
    for key in _fout_tf.keys():
        _diffs_dict[key] = calc_diff(_fout_tf[key], _fout_np[key], name=key)

-----------------------------------
NetWeights: (0, 0, 0, 0, 0, 0)
xf_ diff = 1.0772878340503667e-06
vf_ diff = 4.042994987685233e-06
sld_ diff = 0.0
-----------------------------------
NetWeights: (1, 1, 1, 1, 1, 1)
xf_ diff = 1.898151026580308e-06
vf_ diff = 5.409741788753308e-06
sld_ diff = 4.5733153797300474e-07
